In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time


In [ ]:
authors=[]
dates=[]
statements=[]
sources=[]
targets =[]

In [ ]:
def scrape_website(page_number):
  page_num=str(page_number)
  URL='https://www.politifact.com/factchecks/list/?page='+page_num
  webpage=requests.get(URL)
  soup=BeautifulSoup(webpage.text,'html.parser') #Get the location of the information
  statement_footer = soup.find_all('footer',attrs={'class':'m-statement__footer'})#location of author and date
  statement_quote=soup.find_all('div',attrs={'class':'m-statement__quote'})#location of the statement
  statement_meta=soup.find_all('div',attrs={'class':'m-statement__meta'})#location of the source
  target = soup.find_all('div',attrs={'class':'m-statement__meter'})#Location of the target (score card)
  #Loop through the statement_footer 

  for i in statement_footer:
    link1=i.text.strip()
    name_and_date=link1.split()
    first_name=name_and_date[1]
    last_name=name_and_date[2]
    fullname=first_name+' '+last_name
    month=name_and_date[4]
    day=name_and_date[5]
    year=name_and_date[6]
    date=month+' '+day+' '+year
    dates.append(date)
    authors.append(first_name)

  #loop through the statement_quote
  for i in statement_quote:
    link2 = i.find_all('a')
    statement_text=link2[0].text.strip()
    statements.append(statement_text)

  #loop through the meta
  for i in statement_meta:
    link3=i.find_all('a')
    source_text = link3[0].text.strip()
    sources.append(source_text)

  #loop through the target
  for i in target:
    link4 = i.find('div', attrs={'class': 'c-image'}).find('img').get('alt')
    targets.append(link4)



In [ ]:
#loopcthrough 'n-1' webpages to scrape the data
n=150

for i in range(1,n):
  scrape_website(i)

In [ ]:
# Create the dataframe 
data = pd.DataFrame(columns = ['author','statement','source','date','target'])
data['author']=authors
data['statement']=statements
data['source']=sources
data['date']=dates
data['target']=targets
#show the data

data
# print(targets)


,author,statement,source,date,target
0,Ciara,Video shows Kenya’s president saying he will “...,Instagram posts,"April 14, 2023",false
1,Ciara,"“Congress, White House, IRS, Feds, Federal Res...",Instagram posts,"April 14, 2023",pants-fire
2,Louis,“Fewer people get hurt playing rugby” than Ame...,Joe Biden,"April 14, 2023",barely-true
3,Ciara,Manhattan District Attorney Alvin Bragg “has b...,Facebook posts,"April 14, 2023",pants-fire
4,Ciara,Elon Musk is offering people the opportunity t...,Viral image,"April 14, 2023",false
...,...,...,...,...,...
4405,Ciara,"Says John F. Kennedy said, ""If a Supreme Court...",Viral image,"September 21, 2020",false
4406,Miriam,“Nearly 2 out of 3 Americans want to reduce le...,NumbersUSA,"September 21, 2020",half-true
4407,Clara,"""Candidate Slotkin had presented herself as a ...",Paul Junge,"September 21, 2020",barely-true
4408,Louis,“There's no court session between now and the ...,Joe Biden,"September 21, 2020",false


In [ ]:
def getBinaryNumTarget(text):
  if text == 'true' or text == 'mostly-true' or text == 'half-true':
    return 1

  else:
    return 0

In [ ]:
def getBinaryTarget(text):
  if text == 'true' or text == 'mostly-true' or text == 'half-true':
    return 'REAL'

  else:
    return 'FAKE'

In [ ]:
data['BinaryTarget']=data['target'].apply(getBinaryTarget)
data['BinaryNumTarget']=data['target'].apply(getBinaryNumTarget)

In [ ]:
data


,author,statement,source,date,target,BinaryTarget,BinaryNumTarget
0,Ciara,Video shows Kenya’s president saying he will “...,Instagram posts,"April 14, 2023",false,FAKE,0
1,Ciara,"“Congress, White House, IRS, Feds, Federal Res...",Instagram posts,"April 14, 2023",pants-fire,FAKE,0
2,Louis,“Fewer people get hurt playing rugby” than Ame...,Joe Biden,"April 14, 2023",barely-true,FAKE,0
3,Ciara,Manhattan District Attorney Alvin Bragg “has b...,Facebook posts,"April 14, 2023",pants-fire,FAKE,0
4,Ciara,Elon Musk is offering people the opportunity t...,Viral image,"April 14, 2023",false,FAKE,0
...,...,...,...,...,...,...,...
4405,Ciara,"Says John F. Kennedy said, ""If a Supreme Court...",Viral image,"September 21, 2020",false,FAKE,0
4406,Miriam,“Nearly 2 out of 3 Americans want to reduce le...,NumbersUSA,"September 21, 2020",half-true,REAL,1
4407,Clara,"""Candidate Slotkin had presented herself as a ...",Paul Junge,"September 21, 2020",barely-true,FAKE,0
4408,Louis,“There's no court session between now and the ...,Joe Biden,"September 21, 2020",false,FAKE,0


In [ ]:
data.to_csv('political_fact_checker.csv')